## Tabellenausdrücke (Table Expressions)

In T-SQL werden folgende Arten von Tabellenausdrücken unterschieden:

- **Derived Table**
    - Eine eingeklammerte Unterabfrage mit Tabellen-Alias
    - Typisch eine Unterabfrage in der FROM-Klausel der äußeren Abfrage
    - ist nicht wiederverwendbar
- **Common Table Expression (CTE)**
    - Eine benannte Unterabfrage in einer WITH-Klausel
    - Mehrere Unterabfragen können damit ohne Verschachtelung vorbereitet werden, sodass die Abfrage insgesamt übersichtlich bleibt.
    - ist nicht wiederverwendbar
- **View**
    - Eine Abfrage, die als View-Datenbankobjekt permanent gespeichert ist
    - Kann abgefragt werden wie eine Tabelle
    - ist wiederverwendbar
- **Inline Table-Valued Function**
    - Einem View ähnliches Objekt in der Datenbank, das beim Aufruf Eingabeparameter bekommen kann.
    - ist wiederverwendbar

## Derived Table

Das folgende Beispiel zeigt die beiden günstigsten Produkte jeder Kategorie, indem die äußere Abfrage den Tabellenausdruck filtert.

In [ ]:
SELECT KategorieID, ProduktID, Produkt, Einzelpreis
FROM (
SELECT ROW_NUMBER() OVER(
	PARTITION BY KategorieID 
	ORDER BY Einzelpreis, ProduktID
	) AS RowNum,
	KategorieID, ProduktID, Produkt, Einzelpreis
FROM Produkt ) AS dt
WHERE RowNum <=2;

## Common Table Expression

Die gleiche Abfrage wie oben, diesmal mit vorangestellter CTE.

In [ ]:
WITH cte AS (        
    SELECT ROW_NUMBER() OVER(
        PARTITION BY KategorieID 
        ORDER BY Einzelpreis, ProduktID
        ) AS RowNum,
        KategorieID, ProduktID, Produkt, Einzelpreis
FROM Produkt )
SELECT KategorieID, ProduktID, Produkt, Einzelpreis
FROM cte
WHERE RowNum <=2;

## View

Was in den vorherigen Beispielen als Derived Table bzw. CTE eingebettet war, wird im folgenden Beispiel als View-Objekt angelegt und dann abgefragt.

In [ ]:
DROP VIEW IF EXISTS dbo.vw_ProdukteNachKategorieUndPreis;
GO
CREATE VIEW dbo.vw_ProdukteNachKategorieUndPreis
AS 
SELECT ROW_NUMBER() OVER(
        PARTITION BY KategorieID 
        ORDER BY Einzelpreis, ProduktID
        ) AS RowNum,
        KategorieID, ProduktID, Produkt, Einzelpreis
FROM Produkt;

In [ ]:
SELECT KategorieID, ProduktID, Produkt, Einzelpreis
FROM dbo.vw_ProdukteNachKategorieUndPreis
WHERE RowNum <=2;

## Inline Table-Valued-Function

Folgende Funktion erlaubt, für eine bestimmte Kategorie eine gewünschte Anzahl Produkte anzuzeigen.

In [ ]:
DROP FUNCTION IF EXISTS dbo.udf_GuenstigeProdukte;
GO
CREATE FUNCTION dbo.udf_GuenstigeProdukte (@Kat AS INT, @Num AS INT) RETURNS TABLE
AS 
RETURN
WITH cte AS (        
    SELECT ROW_NUMBER() OVER(
        PARTITION BY KategorieID 
        ORDER BY Einzelpreis, ProduktID
        ) AS RowNum,
        KategorieID, ProduktID, Produkt, Einzelpreis
    FROM Produkt
    WHERE KategorieID = @Kat )
SELECT KategorieID, ProduktID, Produkt, Einzelpreis
FROM cte
WHERE RowNum <= @Num;

In [ ]:
-- Die 5 günstigsten Produkte der 3. Kategorie
SELECT * FROM udf_GuenstigeProdukte(3, 5);

In [ ]:
-- Die 3 günstigsten Produkte der 4. Kategorie
SELECT * FROM udf_GuenstigeProdukte(4, 3);

# Übungen zu Tabellenausdrücken

1. Schreiben Sie die Tabellenwertfunktion **udf\_GuenstigeProdukte** um, so dass anstelle der KategorieID die Bezeichnung der Kategorie als Parameter übergeben wird.
    
    - Entweder verwenden Sie einen JOIN zwischen Produkt und Kategorie
    - Oder Sie nutzen eine Unterabfrage, um die KategorieID zur Bezeichnung zu finden
2. Schreiben Sie ein VIEW, das die Umsätze der Kategorien nach Jahren auflistet.
    
3. Schreiben Sie eine Tabellenwertfunktion, der Sie als Parameter eine Jahreszahl übergeben und die die Gesamtumsätze aller Kategorien im entsprechenden Jahr anzeigt.
    
4. **Delikatessen** letzte Aufgabe in Notebook 02-Kunden: Zeige für jedes Land die beiden Kunden mit dem höchsten Umsatz 2013 an.